In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Memuat file drug200.csv menjadi pandas dataframe
dataframe = pd.read_csv('../input/drug-classification/drug200.csv')

In [ ]:
# Menampilkan 5 baris pertama dari dataframe
dataframe.head()

In [ ]:
# Menampilkan informasi dari dataframe
dataframe.info()

# Mengubah Data Menjadi Numerik
Pada kolom Dtype terdapat banyak jenis data yang berbeda beda, yaitu int64, float64 dan object. Data dengan tipe objek ini merupakan categorical data, sehingga setelah ini harus dilakukan preprocessing untuk mengubah data menjadi numerik atau angka. Karena komputer hanya dapat membaca data yang berupa angka. Adapun caranya kita dapat menggunakan Encoder dari module Sklearn

In [ ]:
dataframe_int = dataframe.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Membuat objek/instance yang bernama encoder
encoder = LabelEncoder()

In [ ]:
categorical_data = ['Sex','BP','Cholesterol','Drug']

Mengubah data pada setiap kolom pada categorical data menjadi numerik/angka. `encoder.fit_transform(argumen)` akan me return data pada argumen yang telah diubah menjadi numerik. Kolom dataframe tersebut akan di replace data numerik yang telah di return

In [ ]:
for kolom in categorical_data:
    dataframe_int[kolom] = encoder.fit_transform(dataframe[kolom])

In [ ]:
# Sekarang data sudah berupa angka sepenuhnya
dataframe_int.head()

In [ ]:
dataframe_int.info()

Mencari tahu berapa banyak kelas yang dimiliki oleh kategori data. Kita akan memprediksi pasien apakah pengonsumsi narkoba jenis Y,C,X,A atau B. Karena terdapat 5 kelas, maka permasalahan ini merupakan multiclass classification

In [ ]:
for kolom in categorical_data:
    print(kolom,dataframe_int[kolom].unique())

In [ ]:
for kolom in categorical_data:
    print(kolom,dataframe[kolom].unique())

Sekarang, kita dapat menganalisa bahwa kelas digantikan oleh angka dan angka tersebut mewakili kelas masing masing. Contohnya pada kolom Sex terdapat dua kelas yaitu 0 dan 1. Dimana 0 mewakili F yaitu Female atau wanita dan 1 mewakili M yaitu Male atau Pria. Begitu juga dengan kolom Drug, terdapat 5 kelas yaitu 0,3,4,1 dan 2. Dimana 0 mewakili Y, 3 mewakili C, 4 mewakili X, 1 mewakili A, dan 2 mewakili B

# Analisis Matrix Korelasi

In [ ]:
# Menampilkan matrix korelasi antar kolom
dataframe_int.corr()

In [ ]:
# Untuk membantu melakukan analisa, akan lebih nyaman jika dilakukan visualisasi data
plt.figure(figsize=(10,8))
plt.title('Matrix Korelasi Data')
sns.heatmap(dataframe_int.corr(),annot=True,linewidths=3)
plt.show()
plt.savefig('Matrix Korelasi Data')

Disini terbaca bahwa kolom Na_to_K memiliki korelasi negatif yang besar sedangkan kolom dengan korelasi positif yang cukup besar hanya pada kolom BP yaitu Blood Pressure atau Tekanan Darah. Sedangkan kolom lain memiliki nilai korelasi yang dekat dengan nol. Artinya, kolom lain tidak terlalu berpengaruh dalam melakukan klasifikasi

# Distribusi Kelas

In [ ]:
def distribusi():
    fig,axes = plt.subplots(nrows=2,ncols=3,figsize=(12,8))
    plt.suptitle('Distribusi',fontsize=24)
    
    def kolom_generator():
        for kolom in dataframe_int:
            yield kolom
    kolom = kolom_generator()

    for i in range(0,2):
        for j in range(0,3):
            k = next(kolom)
            dataframe_int[k].plot(kind='hist',ax=axes[i,j])
            axes[i,j].set_title(k)
    plt.savefig('Distribusi Data.png')
    plt.show()

In [ ]:
distribusi()

Kita dapat menganalisa pada visualisasi Drug dimana frekuensi setiap kelas tidak sama dan memiliki selisih yang jauh. Dimana kelas 0 dan 4 memiliki frekuensi diatas 60 semetara untuk kelas 1,2 dan 3 hanya memiliki frekuensi sekitar 20. Ini menyebabkan Machine Learning model yang akan kita buat setelahnya dapat memprediksi data pada frekuensi yang banyak dengan baik, sementara kesulitan meprediksi data dengan frekuensi sedikit. Karena model cukup berlatih pada kelas dengan frekuensi besar sementara program kurang berlatih pada frekuensi sedikit

# Memisahkan Data
Data harus dipisahkan dari target/label yang ingin diprediksi. Dalam kasus ini, data yang ingin diprediksi merupakan kolom Drug, maka kita pisahkan data menjadi dua. Setelah itu data harus dipisahkan menjadi data latihan dan data test

In [ ]:
data = dataframe_int.drop('Drug',axis=1)
label = dataframe_int['Drug']

In [ ]:
data.head()

In [ ]:
label.head()

In [ ]:
# Kita dapat memisahkan data menjadi data latihan dan data tes dengan train_test_split yang terdapat pada module Sklearn
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data,label,test_size=0.2)

In [ ]:
print(x_train.shape,y_train.shape)

In [ ]:
print(x_test.shape,y_test.shape)

Sekarang data telah terpisah menjadi empat. Dimana data terdapat 160 data untuk latihan dan 40 data untuk test. Dimana jumlah 40 ini berasal dari argumen `test_size=0.2` dimana 40 merupakan 20% dari 200

# Membuat Machine Learning Model
Untuk menyelesaikan masalah klasifikasi atau classification maka kita dapat memilih beberapa pilihan. Dokumentasi dari sklearn https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Membuat objek/instance dengan nama model
model = SVC(gamma='scale')

In [ ]:
# Melatih model dengan data latihan
model.fit(x_train,y_train)

In [ ]:
# Membuat prediksi terhadap data tes
prediction = model.predict(x_test)

In [ ]:
prediction

In [ ]:
# Menampilkan akurasi prediksi model
model.score(x_test,y_test)

# Evaluasi Model

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,r2_score

In [ ]:
def display_conf(y_test,prediction,filename):
    sns.heatmap(confusion_matrix(y_test,prediction),annot=True,linewidths=3,cbar=False)
    plt.title('Confusion Matrix')
    plt.ylabel('Actual')
    plt.xlabel('Prediction')
    plt.savefig(filename)
    plt.show()

In [ ]:
display_conf(y_test,prediction,'ConfusionMatrix-0.png')

Karena label tidak memiliki jumlah kelas yang seimbang, maka kita tidak boleh menilai kualitas model hanya dari akurasinya saja. Namun harus mempertimbangkan recall dan f1 score. Pada confusion matrix terlihat bahwa model bekerja dengan buruk pada kelas 1,2 dan 3 karena data tersebut sangat sedikit jika dibandingkan kelas 0 dan 4

In [ ]:
r2_score(y_test,prediction)

In [ ]:
print(classification_report(y_test,prediction))

# Meningkatkan Model

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
SVC(gamma='scale').get_params()

Untuk mengatasi performa model yang buruk dikarenakan tidak seimbangnya frekuensi setiap kelas, maka kita dapat meningkatkan model dengan cara tuning hyperparameter. Pada estimator SVC terdapat banyak parameter yang dapat dituning, namun kita akan mentuning parameter C dan gamma. Dalam variabel param_grid. Selain dengan tuning hyperparameter, kita juga melakukan cross validation. Dimana kita akan menggunakan 5 fold pada argumen cv pada GridSearchCV()

In [ ]:
param_grid = {'C':[0.01,0.1,1,10,100],
              'gamma':[100,10,1,0,1,0.01]}

In [ ]:
best_model = GridSearchCV(SVC(),param_grid,cv=5,refit=True)

In [ ]:
best_model.fit(x_train,y_train)

In [ ]:
# Model dengan parameter terbaik
best_model.best_estimator_

In [ ]:
# Membuat prediksi dengan model yang telah ditingkatkan
prediction = best_model.predict(x_test)

In [ ]:
# Menampilkan confusion matrix pada prediksi yang baru
display_conf(y_test,prediction,'ConfusionMatrix-1.png')

In [ ]:
r2_score(y_test,prediction)

In [ ]:
print(classification_report(y_test,prediction))

In [ ]:
# Menampilkan data yang sebenarnya
encoder.inverse_transform(np.array(y_test))

In [ ]:
# Menampilkan data yang diprediksi oleh model
encoder.inverse_transform(prediction)

Kita dapat melihat bahwa model dapat memprediksi data dengan baik. Bahkan nyaris memprediksi semuanya dengan benar. Namun dalam pemrograman machine learning, tidak ada model yang dapat memiliki akurasi 100%, dan jika ada maka model itu overfiting. Setelah ini, jika Anda sudah cukup puas dengan peforma model, Anda dapat menggunakan model untuk bekerja. Untuk mensimulasikan bagaimana kita membuat prediksi dengan data baru yang kita input, maka dapat dilakukan sebagai berikut.

In [ ]:
# Menampilkan format data
x_train.head(1)

In [ ]:
def self_prediction():
    age = input('Age : ')
    sex = input('Sex : ')
    bp = input('BP : ')
    chol = input('Cholesterol : ')
    NatoK = input('Na_to_K : ')
    
    # data harus berbentuk (1,5) yaitu [[age,sex,bp,chol,NatoK]]
    print('\nPrediction')
    print('Patient consumed : ',encoder.inverse_transform(best_model.predict([[age,sex,bp,chol,NatoK]]))[0])

In [ ]:
self_prediction()

# Menyimpan Model
Model dapat disimpan dengan `pickle.dump(model,nama_file)`. Model disimpan agar dapat digunakan di lain waktu dan dalam program/script yang berbeda tanpa harus membuat dan melatih model. Untuk memuat model yang telah disimpan tadi dapat dengan funsi `pickle.load(nama_file)` dan akan me-return objek yang di dump. Untuk mengkonfirmasi bahwa model telah dimuat, dapat dilakukan dengan menjalankan methodnya. Disini saya menjalankan `best_estimator_` dan berhasil, yang artinya model berhasil dimuat

In [ ]:
import pickle

In [ ]:
with open('AI_DrugClassifier.pkl','wb') as file:
    pickle.dump(best_model,file)

In [ ]:
with open('AI_DrugClassifier.pkl','rb') as file:
    ml_model = pickle.load(file)

In [ ]:
ml_model.best_estimator_

# Kaggle Submission

In [ ]:
submission = pd.DataFrame()
submission['Actual'] = y_test
submission['Prediction'] = prediction

In [ ]:
submission.head()

In [ ]:
submission.to_csv('Submission.csv')

# Resolusi

Sekarang kita memiliki AI untuk membantu mendiagnosa pasien pengonsumsi narkoba. AI dapat memprediksi jenis narkoba yang dipakai pasien, sehingga model ini dapat digunakan untuk membantu rumah sakit atau badan rehabilitasi dalam mengkategorikan pasiennya. Model ini dapat digunakan sebagai cadangan, jika dokter sedang tidak tersedia atau menangani pasien lain.

Namun perlu diingat bahwa data di dunia nyata tidaklah sesimpel ini. Akan ada banyak data yang kosong dan tidak lengkap, serta untuk proses analisis dan preprocessing data pun memerlukan waktu dan tenaga tersendiri. Karena project ini hanya untuk latihan bagi pemula, maka tidak masalah. Anda hanya perlu terus berlatih dan pada dasarnya semua jenis ilmu yang ada di dunia ini dapat dipelajari.

# Tentang Saya
**Philip Purwoko** | Artificial Intelligence Enthusiast | Mahasiswa di Universitas Sebelas Maret — Surakarta | Indonesia

**Github** : https://github.com/PhilipPurwoko

**Medium** : https://medium.com/@philippurwoko

**Instagram** : https://instagram.com/philippurwoko